### **Traffic Sign Detection - Project**

- **Author**: Uyen Nguyen
- **Date**: 2023/09/29
- **Course**: AI Vietnam - Course 2023
- **Module**: Machine Learning

#### **I. Introduction**

***Traffic Sign Dectection*** is a problem applied algorithms related to the field of Object Detection to 

In [17]:
# Import necessary library
import time
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from skimage.transform import resize
from skimage import feature
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [20]:
# Declare the path to the two images and annotations folder
annotations_dir = "../data/traffic_sign_detection/annotations"
img_dir         = "../data/traffic_sign_detection/images"

# Create two emmpty lists to store images and labels 
label_lst = []
img_lst  = []

Next, we will browse through each .xml file in **annotations** folder. To browser every file name in a folder, we will use `os.listdir()` function. To create a complete path to the .xml file, we use `os.path.join(path1, path2)` to connect folder annotations and file name together. 

In [21]:
for xml_file in os.listdir(annotations_dir):
    xml_filepath = os.path.join(annotations_dir, xml_file)

    tree = ET.parse(xml_filepath)
    root = tree.getroot()

    folder = root.find("folder").text
    img_filename = root.find("filename").text
    img_filepath = os.path.join(img_dir, img_filename)

    # Load the image using OpenCV
    img = cv2.imread(img_filepath)

    for obj in root.findall("object"):
        classname = obj.find("name").text
        if classname == "trafficlight":
            continue

        xmin = int(obj.find("bndbox/xmin").text)
        ymin = int(obj.find("bndbox/ymin").text)
        xmax = int(obj.find("bndbox/xmax").text)
        ymax = int(obj.find("bndbox/ymax").text)

        object_img = img[ymin:ymax, xmin:xmax]
        img_lst.append(object_img)
        label_lst.append(classname)

Examining a sample .xml file in the annotations folder, we can see the following information:

```
<annotation>
    <folder>images</folder>
    <filename>road0.png</filename>
    <size>
        <width>267</width>
        <height>400</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>trafficlight</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>98</xmin>
            <ymin>62</ymin>
            <xmax>208</xmax>
            <ymax>232</ymax>
        </bndbox>
    </object>
</annotation>
```

Inside a .xml file, we will get multiple information about the picture, and the most important information that we want to focus on this project is coordination information and class name of the object (in this case is the traffic sign). So that, we will care about <**name**> and <**bndbox**> entities. Inside a <*object*> entity, <*name*> corresponds to its class and <*bnd*> gives information about its location (coordinate) in the picture. To read the content of a .xml file in Python, we can use this xml module:

In [22]:
print("Number of objects:", len(img_lst))
print("Class names", list(set(label_lst)))

Number of objects: 1074
Class names ['crosswalk', 'stop', 'speedlimit']


In [ ]:
def preprocess_img(img):
    if len(img.shape) > 2:
        img = cv2.cvtColor(
            img,
            cv2.COLOR_BGR2GRAY
        )
    img = img.astype(np.float32)

    resized_img = resize(
        img,
        output_shape  = (32, 32),
        anti_aliasing = True
    )

    hog_feature = feature.hog(
        resized_img,
        orientations = 9,
        pixels_per_cell = (8, 8),
        cells_per_block = (2, 2),
        transform_sqrt = True,
        block_norm = "L2",
        feature_vector = True
    )

    return hog_feature

In [23]:
img_features_lst = []

for img in img_lst:
    hog_feature = preprocess_img(img)
    img_features_lst.append(hog_feature)

img_features = np.array(img_features_lst)

In [24]:
print("Shape of the first image before preprocessing:", img_lst[0].shape)
print("Shape of the first image after preprocessing:", img_features[0].shape)

Shape of the first image before preprocessing: (42, 41, 3)
Shape of the first image after preprocessing: (324,)
